In [ ]:
import scanpy as sc
import numpy as np
import os
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.stats import entropy, itemfreq
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
DATASETS = {
    "pancreas": {"name": "pancreas", "batch_key": "study", "cell_type_key": "cell_type"},
    "brain": {"name": "mouse_brain", "batch_key": "study", "cell_type_key": "cell_type"}
}

In [ ]:
sc.settings.autosave = True

# Pancreas

In [ ]:
data_dict = DATASETS['pancreas']
data_name = data_dict['name']
batch_key = data_dict['batch_key']
cell_type_key = data_dict['cell_type_key']

adata = sc.read(f"./data/{data_name}_normalized.h5ad")
adata

In [ ]:
batches = ["Pancreas inDrop", "Pancreas CelSeq2", "Pancreas CelSeq", "Pancreas Fluidigm C1", "Pancreas SS2"]

In [ ]:
for s in batches:
    batch_adata = adata[adata.obs[batch_key] == s, :]
    batch = pd.DataFrame(data=batch_adata.X.transpose(),
                  index=batch_adata.var_names,
                  columns=batch_adata.obs_names)
    batch.to_csv(f"./{s}.csv")

In [ ]:
os.makedirs(f"./results/Seurat/{data_name}/", exist_ok=True)

### Now run the Seurat-FI.R script, then continue
<br>

In [ ]:
X = pd.read_csv(f'./results/Seurat/{data_name}/corrected.csv', index_col=0).values.T
        
final_adata = None
for s in batches:
    batch_adata = adata[adata.obs[batch_key] == s, :]
    
    if final_adata is None:
        final_adata = batch_adata
    else:
        final_adata = final_adata.concatenate(batch_adata)

final_adata.X = X
sc.tl.pca(final_adata, svd_solver="arpack", n_comps=20)
final_adata = sc.AnnData(X=final_adata.obsm['X_pca'], obs=final_adata.obs)


print(f"after")
sc.pp.neighbors(final_adata)
sc.tl.umap(final_adata)
sc.settings.figdir = f"./results/Seurat/{data_name}/after"
sc.pl.umap(final_adata, color=[batch_key, cell_type_key], wspace=.5)
final_adata.write(f"./results/Seurat/{data_name}/result_adata.h5ad")



# Mouse Brain

In [ ]:
data_dict = DATASETS['brain']
data_name = data_dict['name']
batch_key = data_dict['batch_key']
cell_type_key = data_dict['cell_type_key']

adata = sc.read(f"./data/{data_name}_normalized.h5ad")
adata

In [ ]:
batches = ["Saunders", "Rosenberg", "Tabula_muris", "Zeisel"]

In [ ]:
os.makedirs(f"./results/Seurat/{data_name}/", exist_ok=True)

### Now run the Seurat-FI.R script, then continue
<br>

In [ ]:
X = pd.read_csv(f'./results/Seurat/{data_name}/corrected.csv', index_col=0).values.T
        
final_adata = None
for s in batches:
    batch_adata = adata[adata.obs[batch_key] == s, :]
    
    if final_adata is None:
        final_adata = batch_adata
    else:
        final_adata = final_adata.concatenate(batch_adata)

final_adata = sc.AnnData(X=X, obs=final_adata.obs)
del X
sc.tl.pca(final_adata, svd_solver="arpack", n_comps=20)
final_adata = sc.AnnData(X=final_adata.obsm['X_pca'], obs=final_adata.obs)


print(f"after")
sc.pp.neighbors(final_adata)
sc.tl.umap(final_adata)
sc.settings.figdir = f"./results/Seurat/{data_name}/after"
sc.pl.umap(final_adata, color=[batch_key, cell_type_key], wspace=.5)
final_adata.write(f"./results/Seurat/{data_name}/result_adata.h5ad")